In [29]:
%env JAX_PLATFORM_NAME=cpu

import jaxtyping  # noqa: F401

%load_ext jaxtyping
# %jaxtyping.typechecker beartype.beartype

env: JAX_PLATFORM_NAME=cpu
The jaxtyping extension is already loaded. To reload it, use:
  %reload_ext jaxtyping


In [30]:
import jax
import jax.numpy as jnp
import equinox as eqx
import optax

from tqdm.notebook import trange

from jaxtyping import Array, Bool, Float

from chaogatenn.chaogate import ChaoGate
from chaogatenn.maps import (
    LogisticMap,
    DuffingMap,
    LorenzMap,
    RosslerMap,
    ChenMap,
    RosslerHyperchaosMap,
)
from chaogatenn.utils import grad_norm

In [31]:
# Training data for the AND gate
X = jnp.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=bool)  # Input combinations
AND_Y = jnp.array([0, 0, 0, 1], dtype=bool)  # AND gate output
OR_Y = jnp.array([0, 1, 1, 1], dtype=bool)  # OR gate output
XOR_Y = jnp.array([0, 1, 1, 0], dtype=bool)  # XOR
NAND_Y = jnp.array([1, 1, 1, 0], dtype=bool)  # NAND
NOR_Y = jnp.array([1, 0, 0, 0], dtype=bool)  # NOR
XNOR_Y = jnp.array([1, 0, 0, 1], dtype=bool)  # XNOR
Y = NOR_Y

In [32]:
# Map = LogisticMap(a=4.0)
# Map = LorenzMap()
# Map = DuffingMap(
#     alpha=1.0, beta=1.0, delta=0.02, gamma=8.0, omega=0.5, dt=0.01, steps=1000
# )
# Map = ChenMap(dt=1e-5, steps=100)
Map = RosslerHyperchaosMap()

In [33]:
Map(2)  # type: ignore

Array(0.04477586, dtype=float32)

In [34]:
DELTA, X0, X_THRESHOLD = jax.random.normal(jax.random.PRNGKey(42), (3,))
chao_gate = ChaoGate(DELTA=DELTA, X0=X0, X_THRESHOLD=X_THRESHOLD, Map=Map)

In [35]:
[chao_gate(x) for x in X]

[Array(0.8321855, dtype=float32),
 Array(0.83245987, dtype=float32),
 Array(0.83245987, dtype=float32),
 Array(0.83268064, dtype=float32)]

In [36]:
@eqx.filter_value_and_grad()
def compute_loss(
    chao_gate: ChaoGate, x: Bool[Array, "batch 2"], y: Bool[Array, "batch"]
) -> Float[Array, ""]:  # noqa: F821
    pred = jax.vmap(chao_gate)(x)
    # binary cross entropy
    return -jnp.mean(y * jnp.log(pred + 1e-15) + (1 - y) * jnp.log(1 - pred + 1e-15))

In [37]:
@eqx.filter_jit
def make_step(
    model: ChaoGate,
    x: Bool[Array, "dim 2"],
    y: Bool[Array, "dim"],  # noqa: F821
    optim: optax.GradientTransformation,
    opt_state: optax.OptState,
) -> (Float[Array, "dim"], ChaoGate, optax.OptState):  # type: ignore  # noqa: F821
    loss, grads = compute_loss(model, x, y)
    updates, opt_state = optim.update(grads, opt_state)
    # jax.debug.print(f"{grads, updates}")
    model = eqx.apply_updates(model, updates)
    return loss, model, opt_state

In [38]:
optim = optax.adabelief(3e-5)
opt_state = optim.init(eqx.filter(chao_gate, eqx.is_inexact_array))

In [39]:
epochs = 5_000

for epoch in trange(epochs):
    loss, chao_gate, opt_state = make_step(chao_gate, X, Y, optim, opt_state)  # type: ignore
    _, grads = compute_loss(chao_gate, X, Y)
    grad_norm_value = grad_norm(grads)
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss}, Grad Norm: {grad_norm_value}")

    if loss < 1e-3:
        break

  0%|          | 0/5000 [00:00<?, ?it/s]

Epoch 0, Loss: 0.5845728516578674, Grad Norm: 0.08247623592615128
Epoch 10, Loss: 0.5845413208007812, Grad Norm: 0.08242148160934448
Epoch 20, Loss: 0.5844966769218445, Grad Norm: 0.08234500139951706
Epoch 30, Loss: 0.5844403505325317, Grad Norm: 0.08224914222955704
Epoch 40, Loss: 0.584373950958252, Grad Norm: 0.08213648200035095
Epoch 50, Loss: 0.5842987895011902, Grad Norm: 0.08200912177562714
Epoch 60, Loss: 0.5842157602310181, Grad Norm: 0.08186845481395721
Epoch 70, Loss: 0.5841255784034729, Grad Norm: 0.08171555399894714
Epoch 80, Loss: 0.5840288400650024, Grad Norm: 0.08155130594968796
Epoch 90, Loss: 0.5839260816574097, Grad Norm: 0.0813763365149498
Epoch 100, Loss: 0.5838174819946289, Grad Norm: 0.08119126409292221
Epoch 110, Loss: 0.5837035179138184, Grad Norm: 0.08099658787250519
Epoch 120, Loss: 0.5835843682289124, Grad Norm: 0.08079272508621216
Epoch 130, Loss: 0.5834605693817139, Grad Norm: 0.0805799588561058
Epoch 140, Loss: 0.5833323001861572, Grad Norm: 0.080358669161

In [40]:
print("\nTrained ChaoGate Parameters:")
print(
    f"DELTA: {chao_gate.DELTA}, X0: {chao_gate.X0}, X_THRESHOLD: {chao_gate.X_THRESHOLD}"
)


Trained ChaoGate Parameters:
DELTA: -2.8364293575286865, X0: 1.8786424398422241, X_THRESHOLD: -1.0623588562011719


In [41]:
[
    (
        bool(x1.item()),
        bool(x2.item()),
        (
            chao_gate.Map(chao_gate.X0 + x1 * chao_gate.DELTA + x2 * chao_gate.DELTA)
            > chao_gate.X_THRESHOLD
        ).item(),
    )
    for x1, x2 in X
]

[(False, False, True),
 (False, True, True),
 (True, False, True),
 (True, True, True)]

In [42]:
pred_ys = jax.vmap(chao_gate)(X)
num_correct = jnp.sum((pred_ys > 0.5) == Y)
final_accuracy = (num_correct / len(X)).item()
print(f"final_accuracy={final_accuracy}")

final_accuracy=0.75
